# Python - Multiprocessing

- Introducing Python: Modern Computing in Simple Packages (p.269)
- [Queue](https://www.youtube.com/watch?v=_TNIlBlV5c0)
- http://outofmemory.cn/code-snippet/2267/Python-duojincheng-multiprocessing-usage-example

In [1]:
import multiprocessing
import os
import time

In [2]:
def do_job(name):
    whoami(name)
    
def whoami(name):
    print(f'Process {os.getpid()} doing {name}')
    
def loopy(name):
    whoami(name)
    start = 1
    stop = 100000
    for i in range(start, stop):
        print(f'Doing {i} of {stop}')
        time.sleep(1)

In [3]:
if __name__ == '__main__':
    whoami('Main program')
    for i in range(4):
        p = multiprocessing.Process(target=do_job, args=(f'loop {i}',))
        p.start()

Process 12526 doing Main program
Process 12531 doing loop 0
Process 12532 doing loop 1
Process 12533 doing loop 2
Process 12534 doing loop 3


In [4]:
if __name__ == '__main__':
    whoami('Main program')
    p = multiprocessing.Process(target=loopy, args=(f'loopy',))
    p.start()  # 開始背景作業
    print('Backgound processing...')
    time.sleep(5)
    p.terminate()  # 提前終止
    print('Stopped')

Process 12526 doing Main program
Process 12535 doing loopy
Doing 1 of 100000
Backgound processing...
Doing 2 of 100000
Doing 3 of 100000
Doing 4 of 100000
Doing 5 of 100000
Stopped


---

## Queue

In [5]:
import multiprocessing as mp
import time
import os

In [6]:
def washer(dishes, output):
    for dish in dishes:
        print(f'[{time.time() - tic:.2f}] Washing dish ({os.getpid()}): {dish}')
        time.sleep(1)
        output.put(dish)
        
def dryer(input):
    name = mp.current_process().name
    while True:
        dish = input.get()
        print(f'[{time.time() - tic:.2f}] {name}({os.getpid()}) - Drying dish: {dish}')
        time.sleep(5)
        print(f'[{time.time() - tic:.2f}] {name}({os.getpid()}) - Done: {dish}')
        input.task_done()  # tell queue that the task is done

In [7]:
if __name__ == '__main__':
    tic = time.time()
    dish_queue = mp.JoinableQueue()
    
    # 逐個加載 Processor 到 dish_queue
    dryer_proc1 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc2 = mp.Process(target=dryer, args=(dish_queue,))
    dryer_proc1.daemon = True  # 會在背景一直等著執行 This must be set before start() is called.
    dryer_proc2.daemon = True  # 總共 2 dryers
    dryer_proc1.start()  # 開始進程 1
    dryer_proc2.start()  # 開始進程 2

    # 開始工作
    dishes = ['apple', 'banana', 'orange', 'salad']
    washer(dishes, dish_queue)  # washer 先逐個洗盤子，洗完的盤子丟到 dish_queue
    print('All wash work done!!!')
    dish_queue.join()  # Block until all items in the queue have been gotten and processed.
    print('All Done!')

[0.02] Washing dish (12526): apple
[1.03] Process-6(12536) - Drying dish: apple
[1.03] Washing dish (12526): banana
[2.03] Process-7(12537) - Drying dish: banana
[2.03] Washing dish (12526): orange
[3.04] Washing dish (12526): salad
All wash work done!!!
[6.04] Process-6(12536) - Done: apple
[6.04] Process-6(12536) - Drying dish: orange
[7.04] Process-7(12537) - Done: banana
[7.04] Process-7(12537) - Drying dish: salad
[11.05] Process-6(12536) - Done: orange
[12.05] Process-7(12537) - Done: salad
All Done!


[**Queue vs JoinableQueue**](https://stackoverflow.com/a/31230329/3744499)

JoinableQueue has methods join() and task_done(), which Queue hasn't.

> **class multiprocessing.JoinableQueue( [maxsize] )**
> 
> JoinableQueue, a Queue subclass, is a queue which additionally has task_done() and join() methods.

If you use `JoinableQueue` then you must call `JoinableQueue.task_done()` for each task removed from the queue or else the semaphore used to count the number of unfinished tasks may eventually overflow, raising an exception.

### Deamon Process

In [8]:
import multiprocessing
import time
import sys

def daemon():
    name = multiprocessing.current_process().name
    print(f'[{time.time() - tic:.2f}] Starting: {name}')
    time.sleep(5)
    print(f'[{time.time() - tic:.2f}] Exiting: {name}')

def non_daemon():
    name = multiprocessing.current_process().name
    print(f'[{time.time() - tic:.2f}] Starting: {name}')
    print(f'[{time.time() - tic:.2f}] Exiting: {name}')

if __name__ == '__main__':
    tic = time.time()
    
    d = multiprocessing.Process(name='daemon',
                                target=daemon)
    d.daemon = True  # 守护进程就是不阻挡主程序退出，自己干自己的

    n = multiprocessing.Process(name='non-daemon',
                                target=non_daemon)
    n.daemon = False

    d.start()
    n.start()

    d.join(1)  # 等 d job 完成才繼續，等待n久就不等了
    print('d.is_alive():', d.is_alive())
    n.join()

[0.02] Starting: daemon
[0.03] Starting: non-daemon
[0.04] Exiting: non-daemon
d.is_alive(): True
[5.03] Exiting: daemon
